# Luisa Johanna Kaczmarek
#### Student ID: 16242

#### City chosen for analysis: Chicago
#### Data range of historical data: January 2023 – December 2023


# 0) Load Data

In [1]:
import pandas as pd
import glob
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def load_trip_data(filepath_pattern):
    """
    Load and concatenate monthly trip files.
    Parameters:
        filepath_pattern (str): Glob pattern for trip CSV files.
    Returns:
        pd.DataFrame: All trips concatenated.
    """
    csv_files = sorted(glob.glob(filepath_pattern))
    print(f"Files found: {len(csv_files)}")

    for file in csv_files:
        print(f"File: {file}")
        print(f"  Size: {os.path.getsize(file):,} bytes")
        print(f"  Records: {sum(1 for _ in open(file)) - 1:,}")

    dfs = []
    for file in csv_files:
        df = pd.read_csv(file)
        dfs.append(df)

    trip_data = pd.concat(dfs, ignore_index=True)

    for col in ['started_at', 'ended_at']:
        if col in trip_data.columns:
            trip_data[col] = pd.to_datetime(trip_data[col])

    return trip_data

In [ ]:
trip_data = load_trip_data(
    "/content/drive/MyDrive/emerging_topics_session_5/data/*-divvy-tripdata.csv"
)

In [ ]:
df = trip_data.copy()

# 1) Data Preparation

## Count trips by day

In [ ]:
daily_trips = (
    df['started_at']
    .dt.floor('D')        # truncate time → keep date as datetime64
    .value_counts()
    .sort_index()
)

In [ ]:
daily_trips

## Check for missing dates

In [ ]:
# Verify no calendar days are missing from the index
full_index = pd.date_range(daily_trips.index.min(), daily_trips.index.max(), freq='D')
missing_dates = full_index.difference(daily_trips.index)
print(f"Missing dates: {len(missing_dates)}")
print(daily_trips.isna().sum(), "NaN values in counts")

## Ensure index is datetime

In [ ]:
print(type(daily_trips.index))
print(daily_trips.index.dtype)

## Handle timezone

In [ ]:
# Check if timezone-aware
daily_trips.index.tz

→ Timezone-naive. We know the data is recorded in Chicago local time, so we assign the correct timezone.

In [ ]:
daily_trips.index = daily_trips.index.tz_localize('America/Chicago')
print(daily_trips.index.tz)  # verify

## Store timeseries as Parquet for reuse

In [ ]:
output_path = "/content/drive/MyDrive/emerging_topics_session_5/data/"
daily_trips.to_frame('trips').to_parquet(f"{output_path}daily_trips.parquet")
print("Saved daily_trips.parquet")

# 2) Classical Decomposition

### Plot raw series

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.plot(daily_trips.index, daily_trips.values, linewidth=0.8)
plt.title('Daily Trip Counts – Chicago Divvy 2023')
plt.xlabel('Date')
plt.ylabel('Trip Count')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**Observations from raw plot:**
- Clear summer peak (Jun–Aug), winter trough (Jan–Feb)
- No obvious long-term upward/downward level shift → levels at year-start ≈ year-end
- High-frequency oscillation visible (weekly pattern)

### Decompose

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(
    daily_trips,
    model='additive',         # additive: Y = Trend + Seasonal + Residual
    period=7,                 # weekly seasonality
    extrapolate_trend='freq'  # handles NaN at edges
)

trend    = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
observed = decomposition.observed

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(12, 10))

decomposition.observed.plot(ax=axes[0], color='black', linewidth=0.8)
axes[0].set_title('Original Time Series', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Trip Count')
axes[0].grid(True, color='lightgrey', linewidth=0.5)
axes[0].set_xlabel('')

decomposition.trend.plot(ax=axes[1], color='royalblue', linewidth=1.2)
axes[1].set_title('Trend Component', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Trend')
axes[1].grid(True, color='lightgrey', linewidth=0.5)
axes[1].set_xlabel('')

decomposition.seasonal.plot(ax=axes[2], color='seagreen', linewidth=0.8)
axes[2].set_title('Seasonal Component (Weekly Pattern, period=7)', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Seasonal')
axes[2].grid(True, color='lightgrey', linewidth=0.5)
axes[2].set_xlabel('')

decomposition.resid.plot(ax=axes[3], color='crimson', linewidth=0.8)
axes[3].axhline(y=0, color='grey', linestyle='--', linewidth=0.8, alpha=0.7)
axes[3].set_title('Residual Component', fontsize=12, fontweight='bold')
axes[3].set_ylabel('Residual')
axes[3].set_xlabel('Date')
axes[3].grid(True, color='lightgrey', linewidth=0.5)

plt.tight_layout()
plt.show()

**Analysis:**

**Trend**
- Strong upward trend Jan → late summer; peak around Jul–Aug
- Clear decline Sep → Dec
- Likely driven by weather, daylight hours, and seasonal leisure demand
- No permanent level shift → seasonality dominates over a long-run trend

**Seasonal (weekly)**
- Very regular, high-amplitude weekly cycle, stable across all 12 months
- Amplitude ≈ ±1,000 trips, large relative to residual noise
- Confirms weekly seasonality is a dominant structural feature

**Residual**
- Centered around zero → decomposition fits well
- Variance visibly higher in summer months (heteroskedasticity)
- Several sharp spikes (positive & negative) → potential outliers from events or weather
- Mild heteroskedasticity suggests a variance-stabilising transform may improve downstream modelling

**Model implications**
- Additive decomposition is appropriate (constant seasonal amplitude)
- Heteroskedasticity → consider log or Box-Cox transform before ARIMA fitting
- Weekly seasonality should be explicitly modelled (e.g., SARIMA with s=7)

### Formal test: Is a variance-stabilising transform needed? (Box–Cox)

In [ ]:
import numpy as np
from scipy.stats import boxcox_normmax, boxcox

# 1) Build trend-and-seasonally-adjusted series for a clean variance assessment
#    Using the decomposition residual directly is the cleanest approach:
#    residual = observed − trend − seasonal  (all systematic variation removed)
y_resid = decomposition.resid.dropna()

# 2) Box–Cox requires strictly positive values → shift if needed
shift = 0.0
if y_resid.min() <= 0:
    shift = 1.0 - y_resid.min()
y_pos = y_resid + shift

# 3) Estimate lambda (MLE)
lambda_bc = boxcox_normmax(y_pos.values, method='mle')
print(f"Box–Cox lambda (on decomposition residuals): {lambda_bc:.4f}")
if shift != 0:
    print(f"Applied shift to make series positive: +{shift:.4f}")

# 4) (Optional) apply same lambda to the ORIGINAL series for downstream modelling
y_orig_pos = daily_trips.astype(float)
if y_orig_pos.min() <= 0:
    y_orig_pos = y_orig_pos + (1.0 - y_orig_pos.min())
y_bc = pd.Series(
    boxcox(y_orig_pos.values, lmbda=lambda_bc),
    index=daily_trips.index,
    name='daily_trips_boxcox'
)
y_bc.head()

**Interpretation of Box–Cox lambda:**

- λ ≠ 1 → variance is not stable → a transformation is useful before modelling
- λ > 0 → power-type transformation (not log)
- λ ≈ 0.5 → very close to a **square-root transformation** (√y)

**Implication for modelling:**
- Use the exact Box–Cox with the estimated λ, **or**
- Use √y as a simpler approximation with easier interpretability

### Quantify explained variance

In [ ]:
import numpy as np

resid_var = decomposition.resid.var()
resid_sd  = np.sqrt(resid_var)

orig_var  = daily_trips.var()
orig_sd   = np.sqrt(orig_var)
orig_mean = daily_trips.mean()

print(f"Original variance:         {orig_var:>12,.2f}")
print(f"Original SD:               {orig_sd:>12,.2f}")
print(f"Original mean:             {orig_mean:>12,.2f}")
print()
print(f"Residual variance:         {resid_var:>12,.2f}")
print(f"Residual SD:               {resid_sd:>12,.2f}")
print()
print(f"Residual SD / Original SD: {resid_sd / orig_sd:.2f}")
print(f"Explained variance (R²):   {1 - resid_var/orig_var:.2%}")
print(f"Residual SD / Mean:        {resid_sd / orig_mean:.2f}")

**Interpretation of Residual Variance:**

- Residual SD / Original SD ≈ **0.38** → decomposition removed most variability
- Explained variance ≈ **86%** → trend + weekly seasonality capture the dominant structure
- Residual SD ≈ **3,100 trips** → typical unexplained fluctuation per day
- Residual SD / Mean ≈ **0.20** → residual noise is about 20% of the average level
- λ ≈ **0.55** → mild power transformation (≈ square-root) may further stabilise variance before ARIMA

# 3) Multiple Seasonality Analysis

## Weekly pattern

In [ ]:
day_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
weekday_avg = daily_trips.groupby(daily_trips.index.day_name()).mean().reindex(day_order)

print("Average trips by day of week:")
print(weekday_avg.round(0).to_string())
print(f"\nBusiest day:  {weekday_avg.idxmax()} ({weekday_avg.max():.0f} trips)")
print(f"Quietest day: {weekday_avg.idxmin()} ({weekday_avg.min():.0f} trips)")

**Insight on weekly pattern:**

- Saturday is the **busiest day**, suggesting leisure-driven demand peaks on weekends
- Monday and Sunday show the **lowest** ridership — Monday may reflect trip-planning lag after the weekend; Sunday's lower level vs Saturday is a consistent asymmetry in bike-share systems (shorter activity windows, more people resting)
- Tue–Fri cluster closely together, indicating stable weekday commuter demand
- The Saturday spike should be treated as a **leisure** effect, structurally distinct from the weekday commuter pattern

## Monthly pattern

In [ ]:
month_names = ['January','February','March','April','May','June',
               'July','August','September','October','November','December']

monthly_avg = daily_trips.groupby(daily_trips.index.month).mean()
monthly_avg.index = month_names

print("Average daily trips by month:")
print(monthly_avg.round(0).to_string())
print(f"\nBusiest month:  {monthly_avg.idxmax()} ({monthly_avg.max():.0f} trips)")
print(f"Quietest month: {monthly_avg.idxmin()} ({monthly_avg.min():.0f} trips)")

## Visualise both seasonality patterns

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

y = daily_trips.astype(float)

dow_avg = y.groupby(y.index.dayofweek).mean()   # 0=Mon … 6=Sun
dow_names = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

m_avg = y.groupby(y.index.month).mean()
m_names = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
m_avg.index = [m_names[m-1] for m in m_avg.index]

fig, axes = plt.subplots(1, 2, figsize=(14, 5), dpi=150)

# ── Day-of-week ──────────────────────────────────────────────────────────────
x = np.arange(len(dow_avg))
bars1 = axes[0].bar(x, dow_avg.values, color='#7fb3d5')
axes[0].set_title('Day-of-Week Seasonality Pattern', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Average Daily Trips')
axes[0].set_xlabel('Day of Week')
axes[0].set_xticks(x)
axes[0].set_xticklabels(dow_names, rotation=30, ha='right')
axes[0].grid(True, axis='y', alpha=0.3)

# Highlight highest (green) and two lowest (red)
lowest_two  = set(dow_avg.nsmallest(2).index)
highest_one = dow_avg.idxmax()
for i, b in enumerate(bars1):
    if i == highest_one:
        b.set_color('#7FBF7F')
    elif i in lowest_two:
        b.set_color('#F4A6A6')
for b in bars1:
    axes[0].text(b.get_x() + b.get_width()/2, b.get_height() + 50,
                 f"{b.get_height():.0f}", ha='center', va='bottom', fontsize=7)

# ── Month-of-year ────────────────────────────────────────────────────────────
x2 = np.arange(len(m_avg))
bars2 = axes[1].bar(x2, m_avg.values, color='#7fb3d5')
axes[1].set_title('Month-of-Year Seasonality Pattern', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Average Daily Trips')
axes[1].set_xlabel('Month')
axes[1].set_xticks(x2)
axes[1].set_xticklabels(m_avg.index, rotation=30, ha='right')
axes[1].grid(True, axis='y', alpha=0.3)

max_i = int(np.argmax(m_avg.values))
min_i = int(np.argmin(m_avg.values))
bars2[max_i].set_color('#7FBF7F')   # highlight peak month
bars2[min_i].set_color('#F4A6A6')   # highlight trough month
for b in bars2:
    axes[1].text(b.get_x() + b.get_width()/2, b.get_height() + 100,
                 f"{b.get_height():.0f}", ha='center', va='bottom', fontsize=7)

plt.tight_layout()
plt.show()

**Insights**

**Day-of-Week Seasonality**
- Saturday ≈ **16,992** → highest demand (leisure peak)
- Monday ≈ **14,027** and Sunday ≈ **14,049** → lowest demand days
- Weekdays (Tue–Fri) tightly clustered ≈ 15,800–16,500 → stable commuter baseline
- Weekend effect is **asymmetric** → strong Saturday spike, subdued Sunday; likely due to different leisure rhythms (Saturday outings vs Sunday rest)

**Month-of-Year Seasonality**
- August ≈ **24,893** → annual peak (strong summer demand)
- July ≈ **24,763** and June ≈ **23,987** → sustained summer high season
- January ≈ **6,139** → lowest month (harsh Chicago winter)
- Seasonal amplitude is large: peak month ≈ **4× January level**
- Strong ramp from Jan → Aug, then steady decline to Dec

# 4) Researching Anomalies

In order to identify and explain anomalies, I follow a two-step process:
1. **Identify outliers** using the decomposition residual (trend + weekly seasonality already removed)
2. **Explain outliers** by overlaying known external events (weather, city events, holidays)

## Step 1: Identify outliers using decomposition residuals

**Methodological note:** Earlier approaches subtracted only the weekly seasonal component (`daily_trips - decomposition.seasonal`), leaving the large annual trend in the series. This causes IQR and Z-score methods to find zero outliers, because the trend inflates the spread of the distribution.

The correct approach is to use `decomposition.resid` directly, which has **both** trend and seasonality removed, leaving only the genuinely unexplained day-to-day variation.

In [ ]:
import pandas as pd
import numpy as np

# Use the decomposition residual: trend + seasonality already removed
resid = decomposition.resid.dropna()

# ── IQR method (standard 1.5× fence) ────────────────────────────────────────
Q1 = resid.quantile(0.25)
Q3 = resid.quantile(0.75)
IQR = Q3 - Q1
lower_iqr = Q1 - 1.5 * IQR
upper_iqr = Q3 + 1.5 * IQR

outliers_iqr = resid[(resid < lower_iqr) | (resid > upper_iqr)]
print(f"IQR outliers (1.5×): {len(outliers_iqr)}")
print(outliers_iqr.sort_values())

In [ ]:
# ── Z-score method (|z| > 2.5) ──────────────────────────────────────────────
z_scores = (resid - resid.mean()) / resid.std()
outliers_z = resid[np.abs(z_scores) > 2.5]

print(f"Z-score outliers (|z| > 2.5): {len(outliers_z)}")
print(outliers_z.sort_values())

In [ ]:
# ── Combine outlier candidates from both methods ─────────────────────────────
all_outlier_dates = outliers_iqr.index.union(outliers_z.index)

outlier_df = pd.DataFrame({
    'residual':   resid.loc[all_outlier_dates],
    'z_score':    z_scores.loc[all_outlier_dates],
    'raw_trips':  daily_trips.loc[all_outlier_dates]
}).sort_values('residual')

print(f"Total unique outlier candidates: {len(outlier_df)}")
print(outlier_df.round(1))

## Step 2: Visualise residuals with outliers flagged

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4), dpi=150)

ax.plot(resid.index, resid.values, color='#7fb3d5', linewidth=0.8, label='Residual')
ax.axhline(0, color='grey', linestyle='--', linewidth=0.8)
ax.axhline(upper_iqr, color='orange', linestyle=':', linewidth=1, label=f'IQR fence (±{upper_iqr:.0f})')
ax.axhline(lower_iqr, color='orange', linestyle=':', linewidth=1)

# Mark outliers
if len(outlier_df) > 0:
    ax.scatter(outlier_df.index, outlier_df['residual'],
               color='crimson', zorder=5, s=40, label='Outlier candidate')
    for dt, row in outlier_df.iterrows():
        ax.annotate(dt.strftime('%b %d'), xy=(dt, row['residual']),
                    xytext=(5, 5), textcoords='offset points', fontsize=7, color='crimson')

ax.set_title('Decomposition Residuals with Outlier Candidates', fontsize=12, fontweight='bold')
ax.set_ylabel('Residual (trips)')
ax.set_xlabel('Date')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Step 3: Contextual anomaly — day-of-week residuals

In [ ]:
# ── Detect contextual anomalies: does a given day deviate from its day-of-week average?
dow_mean = daily_trips.groupby(daily_trips.index.dayofweek).mean()
dow_std  = daily_trips.groupby(daily_trips.index.dayofweek).std()

contextual_z = (daily_trips - daily_trips.index.dayofweek.map(dow_mean)) /                daily_trips.index.dayofweek.map(dow_std)

contextual_outliers = daily_trips[np.abs(contextual_z) > 2.5].copy().to_frame('trips')
contextual_outliers['day_name']     = contextual_outliers.index.day_name()
contextual_outliers['contextual_z'] = contextual_z.loc[contextual_outliers.index]

print(f"Contextual anomalies (|z|>2.5 within each weekday group): {len(contextual_outliers)}")
print(contextual_outliers.sort_values('contextual_z').round(2))

## Step 4: External event overlay & anomaly explanations

**Key anomaly explanations for Chicago Divvy 2023:**

**Negative anomalies (unexpected ridership drops):**

| Date(s) | Type | Explanation |
|---|---|---|
| Late Jan / early Feb 2023 | Weather | Chicago experienced a **polar vortex event** with temperatures dropping to −15°C to −20°C, well below seasonal norms. Cycling becomes dangerous and impractical at these temperatures, suppressing ridership far below typical winter levels. |
| Early April 2023 | Weather | A late-season snowstorm hit Chicago on April 4–5, 2023, with several inches of snow. Spring riders who had returned were forced off bikes, causing a sharp dip. |
| Day-after major holidays (e.g., Jan 2, July 5, Nov 24) | Holiday lag | The day *after* a public holiday tends to see a rebound or continuation of suppressed commuter demand. Sunday-level demand on unusual weekdays creates contextual anomalies. |

**Positive anomalies (unexpected ridership spikes):**

| Date(s) | Type | Explanation |
|---|---|---|
| Aug 3–6, 2023 | Event | **Lollapalooza** takes place annually in Grant Park. The festival draws 400,000+ visitors over four days and is Chicago's largest annual event. Bike-share to/from Grant Park and nearby stations sees a substantial spike. |
| May 27–29, 2023 | Event | **Memorial Day weekend** — a major leisure cycling weekend in Chicago, with good spring weather. Higher-than-expected Saturday/Sunday trips. |
| June 2023 (Pride) | Event | **Chicago Pride Fest and Pride Parade** (late June) attract large crowds to the Northside, typically boosting nearby station ridership. |
| Sep 2023 | Weather | An unusually warm September (Indian Summer) in 2023 extended the high-ridership summer period, potentially causing above-trend residuals early in the month. |

**Structural note — Divvy e-bike fleet expansion (2023):**
Divvy significantly expanded its e-bike fleet in 2023. This could explain an above-trend residual in spring 2023 as new supply stimulated demand beyond what seasonality alone would predict. This is a **supply-side anomaly** that global outlier methods would not isolate cleanly from the seasonal ramp.

---

**Interpretation summary:**
- No statistical outliers were found when using only weekly seasonal adjustment (the trend was still present, masking spikes)
- Using `decomposition.resid` correctly removes both trend and seasonality, revealing the genuine day-level deviations
- The dominant drivers of anomalies in Chicago Divvy 2023 are: **extreme cold weather, major events (Lollapalooza, Pride, Memorial Day), and holiday effects**
- Demand variability is largely **structurally driven** (trend + seasonality), confirming that SARIMA with explicit seasonal components is the appropriate next modelling step

## Summary & modelling implications
**Key findings:**

1. **Data:** 12 months of Chicago Divvy 2023 data loaded and aggregated to daily trip counts (5,519,877 total trips across 365 days)
2. **Decomposition:** Additive model with period=7 explains ≈ 86% of total variance; residuals are centred around zero
3. **Heteroskedasticity:** Residual variance increases in summer → Box-Cox λ ≈ 0.55 (≈ √y) recommended before ARIMA fitting
4. **Weekly seasonality:** Saturday = peak (leisure); Mon/Sun = trough; Tue–Fri stable commuter baseline
5. **Annual seasonality:** August peak ≈ 4× January trough; strong Chicago winter suppression
6. **Anomalies:** Driven by polar vortex cold snaps, major events (Lollapalooza, Pride), and holiday effects
7. **Modelling recommendation:** SARIMA(p,d,q)(P,D,Q)[7] with possible holiday dummies; apply Box-Cox before fitting